# Visualização dos pagamentos da Prefeitura

A proposta desse notebook é gerar uma visualização dos pagamentos. A proposta é muito inspirada no [Painel COFOG, do Tesouro Federal](https://painel-cofog.tesouro.gov.br/dashboard_cofog.Rmd#section-especifica%C3%A7%C3%A3o-das-despesas---a%C3%A7%C3%A3o-de-governo).

No entanto, o uso desse notebook tá **bloqueado** porque os dados obtidos pelo Portal da Transparência da Prefeitura apresentam alguns problemas e inconsistência com os dados obtidos pelo TCM-BA. Cabe comparar os dados entre si e com relatórios / balanços da Prefeitura, pra ver o que que a gente pode ter de dado confiável.

Alguns problemas dos dados neste notebook (provenientes do Portal de Transparência da Prefeitura):
- Entradas duplicadas - pagamentos duplicados.
- Entradas negativas - pagamentos negativos; descobri que os pagamentos negativos são utilizados pra "cancelar" pagamentos anteriores, talvez cadastrados de forma errada. O efeito é que no saldo cancela pagamentos registrados talvez de forma indevida, mas no isso causa uma série de outros prejuízos nas análises, por ex: quantos itens foram realmente comprados?
- Falta um valor grande do orçamento quando todos os valores são somados, se comparados aos dados do TCM. Seria bacana uma análise pra ver qual a porcentagem da diferença 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Carregando e limpando os dados

In [ ]:
raw_pagamentos = pd.read_csv('pagamentos.csv')
raw_pagamentos.info()
print(raw_pagamentos.nunique())
raw_pagamentos.head()

In [ ]:
total_entries = len(raw_pagamentos)
raw_pagamentos = raw_pagamentos.drop_duplicates()
uniques_count = len(raw_pagamentos)
print(f'Duplicate entries count: {total_entries - uniques_count} \
\nUnique entries count: {uniques_count}')

In [ ]:
# A quem pertence esse CNPJ?

misterious_cnpj = raw_pagamentos[raw_pagamentos['cpf_ou_cnpj'] == '40.435.740/0001-75']
misterious_cnpj['fonte_recurso'].unique()

In [ ]:
raw_pagamentos = raw_pagamentos[raw_pagamentos['fase'] == 'PAGAMENTO']

In [ ]:
from scripts.parsers import currency_to_float
raw_pagamentos['valor'] = raw_pagamentos['valor'].apply(currency_to_float)

In [ ]:
# Selecting year
raw_pagamentos['data_pagamento'] = pd.to_datetime(raw_pagamentos['data_pagamento'], dayfirst=True)
raw_pagamentos = raw_pagamentos[raw_pagamentos['data_pagamento'].isin(pd.date_range(start='2018-01-01', end='2018-12-31'))]

earlier_date = raw_pagamentos['data_pagamento'].min()
latest_date = raw_pagamentos['data_pagamento'].max()
print(f'Earlier date: {earlier_date}, latest date: {latest_date}')
raw_pagamentos

In [ ]:
payments = raw_pagamentos.drop(['data_publicacao', 'fase', 'numero', 
                'cpf_ou_cnpj', 'data_pagamento', 'numero_processo',
                'credor',  'bem_ou_servico_prestado', 'acao',
                'subfuncao', 'processo_licitatorio', 'fonte_recurso'],
                 axis=1)

In [ ]:
print(payments.info())
print(payments.nunique())
payments.head()

## Conferindo pré-processamento

Antes de passar pra etapa de análise, é importante conferir se os dados fazem sentido. Uma das formas é utilizando a análise descritica do Pandas.

In [ ]:
payments['valor'].describe()

Valores negativos fazem sentido pra pagamentos?

Vamos tentar ter uma ideia melhor do que sejam estes pagamentos negativos:

In [ ]:
negative_payments = raw_pagamentos[raw_pagamentos['valor'] < 0]
print(negative_payments.info())
negative_payments

Munidos dos numeros de processo, vamos ao [Portal da Transparência](http://www.transparencia.feiradesantana.ba.gov.br/index.php?view=despesa) conferir o que são estes valores.

O primeiro, do Banco do Brasil, constam no mesmo dia 3 pagamentos: este negativo, outro de mesmo valor e positivo, outro de valor parecido e positivo. 

Será que quem registrou, registrou primeiro um valor errado, depois um valor igual e negativo (pra anular o errado) e depois um outro valor, dessa vez correto? Vamos olhar os outros pra ver se seguem este padrão

O segundo, do IUNE -INST. DE. URO E NEFRO FSA, existem 10 pagamentos similares (5 de cada valor). Ao final, no balanço da conta, resta apenas um pagamento de cada valor. Parece corroborar com a hipótese anterior.

O terceiro, de SIMOES ESTRELAS EMPREENDIMENTOS LTDA ME EPP, também segue o mesmo padrão. 5 pagamentos que, somados, correspondem a 1 pagamento de um valor.

Vou considerar a hipótese como correta após estas 3 análises pra partir pra análise dos dados, o que não é ideal. No entanto, se cada um que ler este notebook analisar +3 credores / CNPJs e fizer um commit aqui incluindo a análise destes, ao final teremos uma análise correta sobre o que aconteceu.

De acordo com esta hipótese, eu posso simplesmente descartar os valores negativos, pq foram usados apenas pra balancear entradas duplicadas ou erradas. Como as entradas duplicadas foram removidas (50.316 delas, inclusive), isto não constitui um problema. 

Seguiremos com as entradas erradas, no entanto, na fé que não alterem muito o resultado. No total, existem apenas 32 pagamentos negativos não repetidos no ano de 2018 (ano em análise). Dos 3 analisados, apenas o pagamento real (o balanço) do BB constituia um valor diferente, não repetido, do pagamento que foi cancelado. No final das contas teremos um pagamento duplicado do BB. Com sorte, foram poucos casos onde isso ocorreu.



In [ ]:
previous_payments_count = len(payments)
payments = payments[payments['valor'] > 0]

negative_payments_count = len(negative_payments)
payments_count = len(payments)

print((payments_count - previous_payments_count == negative_payments_count))
print((payments_count == previous_payments_count))

In [ ]:
sum = payments['valor'].sum()
highest_payment_percentage = (payments['valor'].max() / sum) * 100

print(f'Payments sum: R$ {sum}')
print(f'Percentage of the biggest payment in relation to all payments: {highest_payment_percentage}%')
payments['valor'].describe()

In [ ]:
payments.quantile([.8, .9, .95, .96, .97, .98, .99, .999])

In [ ]:
payment_threshold = 1_000_000
big_payments = payments[payments['valor'] > payment_threshold]
big_payments_sum = big_payments['valor'].sum()
big_payments_percentage = (big_payments_sum / sum) * 100
print(f'Percentage of payments above R$ {payment_threshold},00: {big_payments_percentage}')

big_payments = big_payments.sort_values('valor', ascending=False)
big_payments

In [ ]:
sorted_payments = payments.sort_values('valor')
index = pd.Index([i for i in range(1, len(sorted_payments) +1)])
sorted_payments.set_index(index, inplace=True)
sorted_payments

In [ ]:
import numpy as np

payments_count = len(sorted_payments)
sorted_payments['cum_sum_ratio'] = (sorted_payments['valor'].cumsum() / sum)
sorted_payments['log_cum_sum_ratio'] = np.log(sorted_payments['cum_sum_ratio'])
sorted_payments['count_percentage'] = sorted_payments.index / payments_count
sorted_payments['price_log'] = np.log(sorted_payments['valor'])

sorted_payments

In [ ]:
sorted_payments.plot.line(x='count_percentage', y='cum_sum_ratio')

In [ ]:
upper_bound = 500000

sorted_payments.plot.line(x='valor', y='cum_sum_ratio')
sorted_payments.plot.line(x='valor', y='cum_sum_ratio', xlim=[0, upper_bound])
sorted_payments.plot.line(x='valor', y='cum_sum_ratio', xlim=[upper_bound, 7400000])

Esses o primeiro plot apresenta uma boa descrição dos pagamentos - falta torná-lo claro.

Os de baixo apresentam coisas interessantes, porém talvez gráfico de linha não seja o melhor jeito de mostrar. Talvez colocando barras em cada faixa de valores (bin) pra mostrar quantos gastos existem naquela faixa de valores. E aí mostrar junto com a faixa do gasto. Aí você tem uma ideia de quanto foi gasto pra aquela faixa de valores, quanto isso representa do total, a quantos pagamentos isto corresponde e quanto o número anterior representa da quantidade total de pagamentos.

ATé minha explicação tá confusa.

Agora os valores parecem fazer sentido.

Existem 19.196 pagamentos

A média dos pagamentos é R$ 2.823,53

O desvio padrão é R$ 13.680,93

O menor valor dos pagamentos é 1 centavo. Ok, essa parte parece ser estranha. Existem 15 pagamentos abaixo de 1 real, 92 abaixo de 5 e 116 abaixo de 10. Parece fazer sentido. Segue o baba

O maior valor é R$ 7.344.698,00

### E depois tem as informações dos quartis: 

25% dos pagamentos são abaixo de R$ 612,48

metade abaixo de R$ 2.500

3/4 abaixo de R$ 12.584,05

# Visualizando os pagamentos

A visualização a seguir se inspira no scatter plot do [Painel COFOG](https://painel-cofog.tesouro.gov.br/dashboard_cofog.Rmd#section-especifica%C3%A7%C3%A3o-das-despesas---a%C3%A7%C3%A3o-de-governo).

Para isso, primeiro são definidos os clusters de acordo com o valor.

Depois, os gastos são separados e plotados por natureza.

- - - -

## Clustering

Primeiro, vamos visualizar os dados pra ver se existe alguma organização, algum insight sobre número de clusters ou qual algoritmo de cluster utilizar.

Estamos clusterizando só por valor, então podemos usar um histograma ou uma curva de densidade pra nos dar uma informação sobre a característica dos dados. Talvez um scatter plot em uma linha também dê alguma intuição sobre isso. Leia mais em: https://www.data-to-viz.com/story/OneNum.html

Primeiro, alguns checks de sanidade pra ver se os valores fazem sentido:

In [ ]:
payments['valor'].plot.hist(bins=100)
# print(f'n: {n}\nBins: {bins}\nPatch: {patch}')

In [ ]:
payments["valor"].plot.hist(bins=100, range=(0,5000), density=True)

In [ ]:
payments["valor"].plot.kde()

In [ ]:
payments.plot.scatter(x='valor', y='valor')

In [ ]:
payments["valor"].plot.box(vert=False)

vou usar um algoritmo ingênuo de cluster: KMeans.

KMeans começa com inicialização aleatória dos centros, então é suceptível a mínimos locais. 

O que se faz é rodar o algoritmo várias vezes e avaliar qual foi a melhor versão e utilizá-la.

In [ ]:
from sklearn.cluster import KMeans